In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/crane_maryland
!mkdir /content/drive/MyDrive/crane_maryland/mask-rcnn-repository

/content/drive/MyDrive/crane_maryland
mkdir: cannot create directory ‘/content/drive/MyDrive/crane_maryland/mask-rcnn-repository’: File exists


In [ ]:
!mkdir /content/drive/MyDrive/crane_maryland/predictions

In [ ]:
!mkdir /content/drive/MyDrive/crane_maryland/polys

In [9]:
%cd /content/drive/MyDrive/crane_maryland/mask-rcnn-repository

/content/drive/MyDrive/crane_maryland/mask-rcnn-repository


In [ ]:
!git clone https://github.com/matterport/Mask_RCNN.git



fatal: destination path 'Mask_RCNN' already exists and is not an empty directory.


In [10]:
#Thanks to https://emadehsan.com/p/object-detection
%cd /content/drive/MyDrive/crane_maryland/mask-rcnn-repository/Mask_RCNN
!python setup.py install 

/content/drive/MyDrive/crane_maryland/mask-rcnn-repository/Mask_RCNN
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'license-file' will not be supported in future versions. Please use the underscore name 'license_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'requirements-file' will not be supported in future versions. Please use the underscore name 'requirements_file' instead
  % (opt, underscore_opt))
running install
running bdist_egg
running egg_info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-level names to mask_rcnn.egg-inf

In [11]:
!pip install 'h5py<3.0.0'

In [12]:
import os
# Root directory of the project
ROOT_DIR = os.path.abspath("./content/drive/MyDrive/crane_maryland/mask-rcnn-repository/Mask_RCNN")

# Import Mask RCNN
!sys.path.append(ROOT_DIR)

/bin/bash: -c: line 0: syntax error near unexpected token `ROOT_DIR'
/bin/bash: -c: line 0: `sys.path.append(ROOT_DIR)'


In [13]:
# Write Annot
def write_annot(obj_type, obj_annots_dir, obj_f_num, org_f_name, org_f_path,org_f_width,org_f_height,org_f_depth, refPts, scores):
    # With credit to: 
    # https://www.geeksforgeeks.org/reading-writing-text-files-python/
    annot_file_path = obj_annots_dir+obj_f_num+'.xml'
    annot_file = open(annot_file_path,"w") 

    annot_file.write("<annotation>\n") 
    annot_file.write("	<folder>"+obj_type+"</folder>\n") 
    annot_file.write("	<filename>"+org_f_name+"</filename>\n") 
    annot_file.write("	<path>"+org_f_path+"</path>\n") 
    annot_file.write("	<source>\n") 
    annot_file.write("		<database>Muhlenberg_charcoal_hearths</database>\n") 
    annot_file.write("	</source>\n") 
    annot_file.write("	<size>\n") 
    annot_file.write("		<width>"+str(org_f_width)+"</width>\n") 
    annot_file.write("		<height>"+str(org_f_height)+"</height>\n") 
    annot_file.write("		<depth>"+str(org_f_depth)+"</depth>\n") 
    annot_file.write("	</size>\n")
    annot_file.write("	<!-- <object_present>0 = the object is not present in the image. 1 = it is -->\n")
    annot_file.write("	<object_present>0</object_present>\n")    
    for ocn in range(0,len(refPts)):
        refPt = refPts[ocn]
        refPtMin = refPt[0]
        refPtMax = refPt[1]
        annot_file.write("	<object>\n") 
        annot_file.write("		<name>"+obj_type+"</name>\n") 
        annot_file.write("		<number>"+str(ocn)+"</number>\n") 
        annot_file.write("		<score>"+str(scores[ocn])+"</score>\n")
        annot_file.write("		<correct>0</correct>\n")
        annot_file.write("		<bndbox>\n") 
        annot_file.write("			<xmin>"+str(refPtMin[0])+"</xmin>\n") 
        annot_file.write("			<ymin>"+str(refPtMin[1])+"</ymin>\n") 
        annot_file.write("			<xmax>"+str(refPtMax[0])+"</xmax>\n") 
        annot_file.write("			<ymax>"+str(refPtMax[1])+"</ymax>\n") 
        annot_file.write("		</bndbox>\n") 
        annot_file.write("	</object>\n") 
    annot_file.write("</annotation>\n") 
    annot_file.close()



In [14]:
%tensorflow_version 1.x
import tensorflow as tf
import keras
import skimage
print(tf.__version__)
print(keras.__version__) # == 2.2.4
#Tensorflow-gpu version #== 1.12.0
print(skimage.__version__) # == 0.14.2


1.15.2
2.3.1
0.16.2


In [ ]:
#Convert tifs to jpgs 
import gdal
import numpy as np
import gdalnumeric
import os

def tif_to_jpg(tif_file, output_file_name, output_folder):
    #print(tif_file,output_file_name, output_folder)
    ds = gdal.Open(tif_file)
    #geoTrans = srcImage.GetGeoTransform()
    band = ds.GetRasterBand(1)
    width = ds.RasterXSize
    height = ds.RasterYSize

    data = band.ReadAsArray(0, 0, width, height)
    #convert all the bad data
    data[data==-9999.0] = 0
    max_value = np.max(data)
    color_multiplier = 255/(max_value-1)
    #print(color_multiplier)
    data = data*color_multiplier
    #print(data)
    data_int = np.array(data, dtype='int')
    #print(data_int)
    #clip = ds.readasarray(ds)
    data_int = data_int.astype(gdalnumeric.numpy.uint8)
    
    print(os.path.join(output_folder,(output_file_name)))
    gdalnumeric.SaveArray(data_int, os.path.join(output_folder,(output_file_name)), format="JPEG")


tif_images_dir_in = '/content/drive/MyDrive/crane_maryland/slope_images/slope_0-199/'
jpg_images_dir_out = '/content/drive/MyDrive/crane_maryland/slope_images/slope_0-199/jpgs/'

if not (os.path.exists(jpg_images_dir_out)):  
    print("Does not exist.")
    os.mkdir(jpg_images_dir_out)
else:
    print(jpg_images_dir_out, " Exists.")

#tif_to_jpg('/content/drive/MyDrive/crane_maryland/predictions/project_2021_nov_13/slope_tifs/slope_20120131_17SQD2093_utm.tif', '17SQD2093_utm.jpg', '/content/drive/MyDrive/crane_maryland/predictions/project_2021_nov_13/jpgs/')

# find all images
counter = 0
for filename in os.listdir(tif_images_dir_in):
    # extract image id
    image_id = filename[:-4]
    ext = filename[-4:]

    if(ext ==".tif"):
        print(counter, filename)
        img_path = os.path.join(tif_images_dir_in, filename)
        tif_to_jpg(img_path, (filename[:28]+".jpg"), jpg_images_dir_out)
        
        counter = counter + 1

# Quality Assurance: Confirm we have 200 slope tifs and 200 DEM slope jpgs with matching file names.

In [30]:

# Dictionary of slope_tifs
slope_tifs_fp = '/content/drive/MyDrive/crane_maryland/slope_images/slope_0-199/'
slope_jpgs_fp = '/content/drive/MyDrive/crane_maryland/slope_images/slope_0-199/jpgs/'

slope_tifs_dict = {}
for file_name in os.listdir(slope_tifs_fp):
    origin_file_path = os.path.join(slope_tifs_fp,file_name)
    #print(origin_file_path)
    if(os.path.isfile(origin_file_path)):  
        if(origin_file_path[-4:]==".tif"):
            # be careful comparing a partial filename
            slope_tifs_dict[file_name[:28]]=file_name


# Dictionary of tifs
slope_jpgs_dict = {}
for file_name in os.listdir(slope_jpgs_fp):
    origin_file_path = os.path.join(slope_jpgs_fp,file_name)
    #print(origin_file_path)
    if(os.path.isfile(origin_file_path)):  
        if(origin_file_path[-4:]==".jpg"):
            # be careful comparing a partial filename
            slope_jpgs_dict[file_name[:28]]=file_name
#print(tifs_dict)

print("There are: ",len(slope_tifs_dict)," tif files in ", slope_tifs_fp)
for dict_key in slope_tifs_dict:
    if not dict_key in slope_jpgs_dict:
        print(dict_key, " (laz) is not in tifs directory.")

print("There are: ",len(slope_jpgs_dict)," jpg files in ", slope_jpgs_fp)
for dict_key in slope_jpgs_dict:
    if not dict_key in slope_tifs_dict:
        print(dict_key, " (tif) is not in jpgs directory.")

prediction_folder_fp = '/content/drive/MyDrive/crane_maryland/predictions/project_2021_nov_14/cfg20200826T2315/unknown/'

xmls_dict = {}
for file_name in os.listdir(prediction_folder_fp):
    origin_file_path = os.path.join(prediction_folder_fp,file_name)
    #print(origin_file_path)
    if(os.path.isfile(origin_file_path)):  
        if(origin_file_path[-4:]==".xml"):
            # be careful comparing a partial filename
            xmls_dict[file_name[:28]]=file_name

print("There are: ",len(xmls_dict)," xml files in ", prediction_folder_fp)
for dict_key in xmls_dict:
    if not dict_key in slope_jpgs_dict:
        print(dict_key, " (xml) is not in jpgs directory.")

#print("There are: ",len(slope_jpgs_dict)," jpg files in ", slope_jpgs_fp)
for dict_key in slope_jpgs_dict:
    if not dict_key in xmls_dict:
        print(dict_key, " (jpg) is not in xmls directory.")


There are:  200  tif files in  /content/drive/MyDrive/crane_maryland/slope_images/slope_0-199/
There are:  200  jpg files in  /content/drive/MyDrive/crane_maryland/slope_images/slope_0-199/jpgs/
There are:  200  xml files in  /content/drive/MyDrive/crane_maryland/predictions/project_2021_nov_14/cfg20200826T2315/unknown/


In [ ]:
# Predict it for all the images
# Run Predict Config and Write Annot before running this cell.
cfg_name='cfg20200826T2315'
model_epoch='0016'
object_type = "charcoal_hearth_hill"

print("cfg_name:",cfg_name)

#For all of the jpegs, run prediction

#images_dir
# Run on all images - produce results for analysis.
# images_dir = '/content/drive/MyDrive/crane_maryland/tif/tif_tiles/jpgs'

# 2021 Nov 13
#images_dir = '/content/drive/MyDrive/crane_maryland/predictions/project_2021_nov_13/jpgs/'

images_dir = '/content/drive/MyDrive/crane_maryland/slope_images/slope_0-199/jpgs/'

import os
from os import listdir
import skimage
import cv2
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes
from mrcnn.config import Config
import mrcnn.model as modellib

# define the prediction configuration
class PredictionConfig(Config):
	# define the name of the configuration
	NAME = object_type+"_cfg" 
	# number of classes (background + class (charcoal hearth))
	NUM_CLASSES = 1 + 1
	# simplify GPU config
	GPU_COUNT = 1
	IMAGES_PER_GPU = 1

# create config
cfg = PredictionConfig()
# define the model

model = modellib.MaskRCNN(mode='inference', model_dir='/content/drive/MyDrive/MaskCNNhearths/code_and_working_data/charcoal_hearth_hill/model/', config=cfg)
print("Model loaded.")
# load model weights
model_path = '/content/drive/MyDrive/MaskCNNhearths/code_and_working_data/charcoal_hearth_hill/model/charcoal_hearth_hill_cfg20200826T2315/mask_rcnn_charcoal_hearth_hill_cfg_0016.h5'
#model_path = '/home/student/charcoal_hearth_hill/model/'+object_type+'_'+cfg_name+'/mask_rcnn_'+object_type+'_cfg_'+model_epoch+'.h5'
model.load_weights(model_path, by_name=True)

image_prediction_folder = '/content/drive/MyDrive/crane_maryland/predictions/project_2021_nov_14/'+cfg_name+'/'

image_for_prediction_folder = images_dir

if not os.path.exists(image_prediction_folder):
    os.mkdir(image_prediction_folder)
image_prediction_folder = image_prediction_folder+'/unknown/'
if not os.path.exists(image_prediction_folder):
    os.mkdir(image_prediction_folder)
    
    
#Store the results in XML    
class_names = [object_type,object_type,]


# find all images
file_counter = 0
for filename in listdir(images_dir):
    # extract image id
    image_id = filename[:-4]
    ext = filename[-4:]
    if(ext ==".jpg"):
        print(file_counter)  
        img_path = os.path.join(images_dir, filename)
        #ann_path = annotations_dir + image_id + '.xml'
        #print(image_id,img_path,ann_path)
        print(image_id,img_path)
    
        image = cv2.imread(img_path)
        #image = skimage.io.imread(os.path.join(image_for_prediction_folder, test_file_names[cn]))
        #print("shape:", image.shape)
        # Run detection

        results = model.detect([image], verbose=1)
        # Visualize results
        r = results[0]
        #print(r['rois'], class_names, r['scores'])
        #print(r['rois'][1],r['rois'][1][0],r['rois'][1][1],r['rois'][1][2],r['rois'][1][3])
        #image2= display_instances(image, r['rois'], r['masks'], r['class_ids'], 
        #class_names, r['scores'])
        obj_f_num = image_id
        refPts = []
    
        for rcn in range(0,len(r['rois'])):
            refPt = []
            refPtM = []
            refPtM.append(r['rois'][rcn][1])
            refPtM.append(r['rois'][rcn][0])
            refPt.append(refPtM)
            refPtM = []
            refPtM.append(r['rois'][rcn][3])
            refPtM.append(r['rois'][rcn][2])
            refPt.append(refPtM)
            refPts.append(refPt)
            #print(refPts)
        write_annot(class_names[0], image_prediction_folder, obj_f_num, filename, image_for_prediction_folder,image.shape[1],image.shape[0],image.shape[2], refPts,r['scores'])
        file_counter = file_counter + 1
print("The results are in: "+image_prediction_folder)

In [ ]:
        image = cv2.imread('/content/drive/MyDrive/crane_maryland/predictions/project_2021_nov_13/slope_tifs/slope_20120203_17SQD1896_utm.tif')
        #image = skimage.io.imread(os.path.join(image_for_prediction_folder, test_file_names[cn]))
        #print("shape:", image.shape)
        # Run detection
        print(image)

        results = model.detect([image], verbose=1)

None
Processing 1 images
image


AttributeError: ignored

In [ ]:
import os
from os import listdir
import cv2
import skimage
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from google.colab.patches import cv2_imshow
from matplotlib import pyplot
from matplotlib.pyplot import figure

#coordinate = "0106"
#training_image = '/content/drive/MyDrive/crane_ane/data/images/jpgs/401'+coordinate +'.jpg'
#prediction_annot_file_path = '/content/drive/MyDrive/MaskCNNhearths/code_and_working_data/charcoal_hearth_hill/predictions/cfg20200826T2315/unknown/401'+coordinate +'.xml'

prediction_image_folder = '/content/drive/MyDrive/crane_maryland/tif/tif_tiles/jpgs/'
prediction_annot_folder = '/content/drive/MyDrive/crane_maryland/predictions/cfg20200826T2315/unknown'

training_annot_file_path = ""

font = cv2.FONT_HERSHEY_SIMPLEX 
font_scale = 1
font_color = (0, 200, 0) 
font_thickness = 2

for prediction_image in listdir(prediction_image_folder):
    print(prediction_image)
    annotation_file = prediction_image[:-4]+".xml"
    image = cv2.imread(os.path.join(prediction_image_folder,prediction_image))
    print(image.shape)
    prediction_annot_file_path = os.path.join(prediction_annot_folder,annotation_file)

    overlay = image.copy()
    output = image.copy()
    #cv2_imshow(output)    
    font_color = (255, 0, 255)
    font_scale = .6
    font_thickness = 1

    print(prediction_annot_file_path)
    if(os.path.exists(prediction_annot_file_path)):
        tree_train = ElementTree.parse(prediction_annot_file_path)
        root_train = tree_train.getroot()
        for obj in root_train.findall('./object'):
            box = obj.find('bndbox')
            xmin = int(box.find('xmin').text)
            ymin = int(box.find('ymin').text)
            xmax = int(box.find('xmax').text)
            ymax = int(box.find('ymax').text)
            box_num = str(obj.find('number').text)
            score = float(obj.find('score').text)
            if(ymin>ymax):
                ytemp = ymin
                ymin = ymax
                ymax=ytemp
            if(xmin>xmax):
                xtemp = xmin
                xmin = xmax
                xmax=xtemp            
            coors = [xmin, ymin, xmax, ymax]
            crop_output = output[ymin:ymax, xmin:xmax]
        
            cv2.rectangle(output, (xmin, ymin), (xmax, ymax),(255, 0, 255), 2)
            cv2.putText(output, f'{score:.3f}' , (xmin+0, ymin-17), font, font_scale, font_color, font_thickness, cv2.LINE_AA)
        print(root_train.findall('./object'))
        objs = root_train.findall('./object')
        if(len(objs)>0):
            cv2_imshow(output)
    else:
        print("No prediction annotation file for this image.  This is likely ok.")




#crop_output = output[ymin:ymax, xmin:xmax]
#resized = cv2.resize(output, (64, 30))